# Innhenting av data fra eksterne datakilder

Som oftest så er dataene du behøver for å gjennomføre ett gitt data analyse prosjekt hentet fra eksterne kilder.
Det er fullt mulig å manuelt laste ned data filene du trenger fra websider og flyte dem til de forskjellige mappene hvor du skal arbeide, men dette er ikke en god metode i det lange løp. Manuelt arbeid som dette må da dokumenters i tilegg til analyse arbeidet og muligheten for feil forstørres.

"Best practice" i data science miljøet er å skrive skript som tar seg av innhetingen av data. Det er mange fordeler med dette. Innheting prosessen er da eksplisit dokumentert i kode og man sparer tid ved gjenntatte innhentinger.


![title](resources/HTTP_request_response.svg)

Vi skal ta i dette prosjektet ta i bruk data filer gjort tilgjengelig av NAV hos  __http://www.data.norge.no__

Vi skal jobbe på data samlet inn om stillingsannonser som har blitt utlyst gjennom NAV i perioden 2002 - 2017

#### I dette prosjektet så skal vi bruke python pakken Requests
Requests er en python pakke som gjør skripting av HTTP baserte operasjoner lett og smertefritt
<br>Dokumentasjon: __http://docs.python-requests.org/en/master/__
<br><br>For mer informasjon ang HTTP som er den grunnleggende protokolen som styrer kommunikasjonen til websider og det fleste tilgjengelige APIer:<br>
__https://developer.mozilla.org/en-US/docs/Web/HTTP/Overview__

Vi kan bruke requests til å gjøre kall til nettsider akkurat som en nettleser gjør.

In [5]:
import requests

#gjør kall til google.com med HTTP GET metoden
result = r = requests.get("https://google.com")

#Serveren returnerer HTTP status kode 200 hvis kallet ble gjennomført uten feil.
r.status_code

200

## Innhenting av CSV datafiler
CSV er en vanlig brukt fil type for data sett og er standard måten å hente ut arbeids utlysing data settet.

In [13]:
# os er en innebygd python pakke som gir oss tigang til å interaktere med operativsystemet.
import os

#URL til API endepunket for data om stillings annonser utlyst i 2017
api_url = "https://hotell.difi.no/download/nav/ledige-stillinger/2017"

#Finner path til data/raw mappen hvor vi legger rå data.
file_name = "arbeids_utlysininger_2017.csv"
data_path = os.path.join(os.path.pardir, "data", "raw")
write_path = os.path.join(data_path, file_name)

#Henter data settet fra API endepunktet med HTTP GET
result = requests.get(api_url)

#Hvis vi kallet var suksessfult og vi har motatt data så lagrer vi data settet i rå data mappen.
if result.status_code == 200:
        with open(write_path, "wb") as handle:
            for block in result.iter_content(1024):
                handle.write(block)
        if os.path.isfile(write_path):
            print(file_name + " er lagret under: " + data_path)
else:
    print("Kunne ikke laste ned data settet | HTTP Response: " + str(result.status_code))


arbeids_utlysininger_2017.csv er lagret under: ../data/raw


## Innhenting av JSON
Vi kan også hente ut data settet i JSON formatet. JSON er et forholdsvis nytt format å transportere data i, men har raskt blitt det vanligste transport formatet av data mellom APIer på nett.

In [50]:
import os
import pandas as pd
import codecs
import json

json_api_url = "https://hotell.difi.no/api/json/nav/ledige-stillinger/2017"


#Finner path til data/raw mappen hvor vi legger rå data.
file_name = "arbeids_utlysininger_2017.json"
data_path = os.path.join(os.path.pardir, "data", "raw")
write_path = os.path.join(data_path, file_name)

#Henter data settet fra API endepunktet med HTTP GET
result = requests.get(json_api_url)

with open(write_path, 'wb') as f:
    for block in result.iter_content(1024):
            f.write(block)

In [58]:
from pandas.io.json import json_normalize 


with open(write_path) as f:
    data = json.load(f)

df = json_normalize(data["entries"])
df.set_index(df.stillingsnummer)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 22 columns):
antall_stillinger             100 non-null object
arbeidssted_fylke             100 non-null object
arbeidssted_fylkesnummer      100 non-null object
arbeidssted_kommune           100 non-null object
arbeidssted_kommunenummer     100 non-null object
arbeidssted_land              100 non-null object
arbeidssted_landkode          100 non-null object
isco_versjon                  100 non-null object
nav_enhet_kode                100 non-null object
offisiell_statistikk_flagg    100 non-null object
registrert_dato               100 non-null object
sistepubl_dato                100 non-null object
statistikk_aar_mnd            100 non-null object
stilling_kilde                100 non-null object
stillingsnummer               100 non-null object
stillingstittel               100 non-null object
virksomhet_navn               100 non-null object
virksomhet_organisasjonsnr    100 non-null obj

,antall_stillinger,arbeidssted_fylke,arbeidssted_fylkesnummer,arbeidssted_kommune,arbeidssted_kommunenummer,arbeidssted_land,arbeidssted_landkode,isco_versjon,nav_enhet_kode,offisiell_statistikk_flagg,...,statistikk_aar_mnd,stilling_kilde,stillingsnummer,stillingstittel,virksomhet_navn,virksomhet_organisasjonsnr,yrke,yrke_grovgruppe,yrkesbetegnelse,yrkeskode
0,1,Østfold,01,Halden,0101,Norge,NO,ISCO-08,0101,1,...,201701,Reg av arb.giver på nav.no,0101201612000013,Pedagogisk leder,YNGLINGEN BARNEHAGE,974122545,Førskolelærere,Undervisning,Barnehagelærer,2342
1,1,Oslo,03,Oslo,0301,Norge,NO,ISCO-08,0101,1,...,201701,Meldt til NAV lokalt,0101201612000014,Bussjåfør,*,*,Bussjåfører og trikkeførere,Reiseliv og transport,Bussjåfør,8331
2,1,Østfold,01,Halden,0101,Norge,NO,ISCO-08,0101,1,...,201701,Overført fra arbeidsgiver,0101201612000015,Rådgiver - studentopptak og internasjonalisering,HØGSKOLEN I ØSTFOLD,973967592,Spesialister i pedagogikk,Undervisning,Studiekonsulent,2351
3,1,Østfold,01,Halden,0101,Norge,NO,ISCO-08,0101,1,...,201701,Overført fra arbeidsgiver,0101201612000016,Undervisnings- og forskerstilling i musikk,HØGSKOLEN I ØSTFOLD,973967592,Universitets- og høyskolelektorer/-lærere,Undervisning,Førsteamanuensis,2310
4,1,Østfold,01,Halden,0101,Norge,NO,ISCO-08,0101,1,...,201701,Reg av arb.giver på nav.no,0101201612000017,Kantinemedarbeider,AXXE AS,973070452,Kjøkkenassistenter,Reiseliv og transport,Anretningsassistent,9412
